In [1]:
import numpy as np
import pandas as pd
import os
import ntpath

In [2]:
file = './data/345_CLNF_gaze.txt'

In [3]:
df = pd.read_table(file, sep=', ', index_col='frame')

C:\Users\65842\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [4]:
df.head()

,timestamp,confidence,success,x_0,y_0,z_0,x_1,y_1,z_1,x_h0,y_h0,z_h0,x_h1,y_h1,z_h1
frame,,,,,,,,,,,,,,,
1,0.000000,0.983915,1,-0.023546,0.242617,-0.969836,-0.220724,0.207532,-0.953001,-0.069815,0.469520,-0.880157,-0.266090,0.429935,-0.862758
2,0.033333,0.984079,1,-0.017832,0.243434,-0.969753,-0.235717,0.215903,-0.947535,-0.058182,0.481466,-0.874531,-0.274890,0.446279,-0.851628
3,0.066667,0.983523,1,-0.020343,0.239940,-0.970574,-0.242104,0.215758,-0.945957,-0.057735,0.484280,-0.873006,-0.278243,0.450642,-0.848235
4,0.100000,0.983979,1,-0.022708,0.235896,-0.971513,-0.248893,0.220147,-0.943180,-0.057406,0.486755,-0.871650,-0.282320,0.459116,-0.842323
5,0.133333,0.983953,1,-0.023246,0.235745,-0.971537,-0.250700,0.220682,-0.942576,-0.056716,0.486648,-0.871755,-0.282910,0.459063,-0.842154


In [13]:
def get_features(file):
    filename = ntpath.basename(file)
    participant_no = int(filename[0:3])
    
    df = pd.read_table(file, sep=', ', index_col='frame')
    features = df.drop(columns=['timestamp'])
    
    return participant_no, features
    
    
def compile_files(directory, label_file):
    column_names_final = ["Participant_ID", "PHQ_Score", "PHQ_Binary"]
    labels = pd.DataFrame(columns = column_names_final)
    
    compiled_features = []
    compiled_labels = []
    min_len = 100000
    
    feature_files = [pos for pos in os.listdir(directory) if pos.endswith('.txt')]
    df_labels = pd.read_csv(label_file, index_col = "Participant_ID")
    
    for filename in feature_files:
        file = directory+'/'+filename
        participant_no, features = get_features(file)
        features = np.array(features)
        
        length = features.shape[0]
        if length < min_len:
            min_len = length
        
        PHQ_Binary = df_labels.loc[participant_no]["PHQ_Binary"]
        PHQ_Score = df_labels.loc[participant_no]["PHQ_Score"]
        new_row = {"Participant_ID": participant_no, "PHQ_Score": PHQ_Score, "PHQ_Binary": PHQ_Binary}
        labels = labels.append(new_row, ignore_index = True)
        
        compiled_features.append(features)
        
    
    compiled_features = np.array(compiled_features)
    
    features = []
    for i in range(compiled_features.shape[0]):
        features.append(compiled_features[i][:min_len][:])
    features = np.array(features)
    
    return features, labels

def average_features(directory, label_file):
    column_names_final = ["Participant_ID", "PHQ_Score", "PHQ_Binary"]
    labels = pd.DataFrame(columns = column_names_final)
    
    compiled_features = []
    compiled_labels = []
    
    feature_files = [pos for pos in os.listdir(directory) if pos.endswith('.txt')]
    df_labels = pd.read_csv(label_file, index_col = "Participant_ID")
    
    for filename in feature_files:
        file = directory+'/'+filename
        participant_no, features = get_features(file)
        
        features = features[features.success != 0]
        features= features.mean()
        features = features.drop(columns=['success', 'confidence'])
        features = np.array(features)
        
        PHQ_Binary = df_labels.loc[participant_no]["PHQ_Binary"]
        PHQ_Score = df_labels.loc[participant_no]["PHQ_Score"]
        new_row = {"Participant_ID": participant_no, "PHQ_Score": PHQ_Score, "PHQ_Binary": PHQ_Binary}
        labels = labels.append(new_row, ignore_index = True)
        
        compiled_features.append(features)
        
    
    compiled_features = np.array(compiled_features)
    
    return compiled_features, labels

In [14]:
directory = './data'
label_file = 'Y_data.csv'

In [7]:
features, labels = compile_files(directory, label_file)

C:\Users\65842\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """
C:\Users\65842\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [8]:
features.shape

(181, 12447, 14)

In [9]:
np.save("gaze_features.npy", features)

In [10]:
labels.to_csv("gaze_labels.csv", index=False)

In [15]:
features, labels = average_features(directory, label_file)

C:\Users\65842\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """


In [16]:
np.save("avg_gaze.npy", features)

In [18]:
features.shape

(181, 14)